In [1]:
import torch
import torch.nn as nn
import logging

from comblearn.data import DSFValueFunction
from comblearn.env import CombinatorialAuction

logging.basicConfig(level=20)

In [11]:
logger = logging.getLogger()
assert len(logger.handlers) == 1
handler = logger.handlers[0]
handler.setLevel(logging.INFO)

In [2]:
device = 'cuda'

class MyModular(nn.Module):
    def __init__(self, k):
        super(MyModular, self).__init__()
        self.w = torch.randint(0, 5, (k, 1)).float().to(device)
        logging.info(f"{self.w.squeeze()}")

    def forward(self, x):
        return torch.matmul(x, self.w)


def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [3]:
N = 5
m = 8
bidders = list(range(N))
q_init = 500
q_max = 505
items = list(range(m))

value_functions = [MyModular(m), 
                   MyModular(m), 
                   MyModular(m), 
                   MyModular(m), 
                   MyModular(m)]

value_functions_l = [DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300), 
                     DSFValueFunction(items, 110, [3], 300)]

INFO:root:tensor([3., 2., 3., 1., 3., 4., 2., 2.], device='cuda:0')
INFO:root:tensor([1., 0., 3., 1., 0., 4., 4., 3.], device='cuda:0')
INFO:root:tensor([4., 4., 2., 0., 1., 3., 1., 1.], device='cuda:0')
INFO:root:tensor([0., 0., 2., 1., 3., 0., 2., 2.], device='cuda:0')
INFO:root:tensor([2., 4., 4., 4., 0., 1., 3., 3.], device='cuda:0')


In [4]:

auction = CombinatorialAuction(bidders, items, value_functions, value_functions_l, q_init, q_max)
allocations, payments = auction.run(epochs=1000, lr=0.001, delta=0.005, sample_rate=5)

INFO:root:Query generation...
INFO:root:Step: 1/1, Query shapes: [torch.Size([500, 8]), torch.Size([500, 8]), torch.Size([500, 8]), torch.Size([500, 8]), torch.Size([500, 8])]
INFO:root:Generating main query...
INFO:root:Bidder 1, loss: 0.17644347250461578
INFO:root:Bidder 2, loss: 0.3633887469768524
INFO:root:Bidder 3, loss: 0.2133006602525711
INFO:root:Bidder 4, loss: 0.6473228931427002
INFO:root:Bidder 5, loss: 0.5775323510169983
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:Main query generated.
INFO:root:Generating marginal queries...
INFO:root:Bidder 1, loss: 0.03771185874938965
INFO:root:Bidder 2, loss: 0.056291498243808746
INFO:root:Bidder 3, loss: 0.20315274596214294
INFO:root:Bidder 4, loss: 0.057892199605703354
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:Marginal query 1 gener